In [1]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as pxK
print(pd.__version__)

1.4.4


In [2]:
# import Excel file from Windows
# fileDir = r'C:\Users\HarleyKing\OneDrive - LuminUltra Technologies Ltd\Documents\Python Scripts\canonical_pcr_data'
# fileDir = r'C:\Users\HarleyKing\OneDrive - LuminUltra Technologies Ltd\Documents - LTI Research & Development\Automation and Assay Dev\Exp800.28_Nitrifiers_for_Lab_Services\Exp800.28.02_arch_amoA'
# fileDir = r'C:\Users\HarleyKing\OneDrive - LuminUltra Technologies Ltd\Documents - LTI Research & Development\Automation and Assay Dev\Exp800.28_Nitrifiers_for_Lab_Services\Exp800.28.02_arch_amoA'
fileDir = r"C:\Users\HarleyKing\OneDrive - LuminUltra Technologies Ltd\Documents - LTI Research & Development\Automation and Assay Dev\Exp800.28_Nitrifiers_for_Lab_Services\Exp800.28.02_arch_amoA"
# fileDir = r'qPCR_data'
# fileName = r'20220708_122234_SRB_dsr_test_2.xls' #cy5 data
# fileName = r'Exp800.05.create.primer.matrix.micH.xlsx' #fam data
# fileName = r'AF Fuel Primer Matrix Run 2 01Jun21 GH.xls' #fam data
# fileName = r'three_fluor_assay.xls' #fam data
# fileName = r'20230124_120012_Exp800.28_a_amoA_stds1_15.xls' #fam data
fileName = r'20230131_104302_Exp800.28_a_amoA_temp_grad.xls' #fam data
# create dataframe
dfExcel = pd.read_excel(fileDir+'\\'+fileName, sheet_name=None, header=None) #get all sheets

# show all sheet names
sheetNames = list(dfExcel.keys())
print(sheetNames)

['Detectors', 'Sample', 'Plate', 'Program', 'Quan. Raw Data', 'Melt Raw Data', 'Quan. Result', 'Melt Result', 'Detail Melt Result', 'Quan. AmpData-SYBR']


In [3]:
class Fluor:
    probe: str
    def __init__(self, probe):
        self.probe = probe
        # self.maxRow = self.getMaxRow()
    def dfCreate(self): # make a df from the excel data
        # find probe in sheet names
        probe_sheet = [sheet for sheet in sheetNames if self.probe in sheet]
        # create dataframe from sheet name
        df = dfExcel[probe_sheet[0]]
        # assign first row to be column header
        df.columns = df.iloc[0]
        # drop the first row (duplicate of header)
        df = df.drop(df.index[0])
        # convert all columns to numberic
        df = df.apply(pd.to_numeric, errors='coerce')
        # make index the first column
        df.set_index('Cycle', inplace=True)
        # fluor values less than 1 get 0
        df = df.where(df>=1, 0)
        return df
    def getMaxRow(self):
        df = self.dfCreate()
        # get max value for each row and convert to list
        maxRow = df.loc[df.index == 40].values.flatten().tolist()
        # round every element in list to one decimal place, convert to str
        maxRowRound = [str(round(x,1)) for x in maxRow]
        return maxRowRound
    def plotAllWells(self):
        df = self.dfCreate()
        # make plot of wells
        fig = px.line(
            df,
            x=df.index,
            y=list(df.columns),
            color_discrete_sequence=px.colors.qualitative.Set3)
        fig.update_layout(
            height=500,
            width=1000,
            xaxis_title='Cycle',
            yaxis_title='RFU ({})'.format(self.probe),
            title_text='Cycle vs Fluorescence ({})'.format(self.probe),
            legend_title='Well')
        return fig.show()
    def plotIndWells(self):
        df = self.dfCreate() 
        maxRow = self.getMaxRow()
        # make subplots of individual wells
        colorsList = px.colors.qualitative.Set3
        figSub = make_subplots(
            rows=8, cols=12,
            shared_xaxes=True,
            shared_yaxes='all',
            start_cell='top-left',
            subplot_titles=list(df.columns)) #, subplot_titles=sub_titles, print_grid=False)
        for r in range (0,8): #loop through rows
            for c in range(0,12):
                figSub.add_trace(go.Scatter(
                    x=df.index,
                    y=df.iloc[:,r+(r*11)+c], # this pattern translates r=8, c=12 to 0..95 columns in df (0 indexed)
                    name=df.columns[r+(r*11)+c], # gets column name eg 'A01'
                    mode="lines+text", # allows for text to be inserted onto ind plot
                    line_color=colorsList[c], # assign color to each well consistent with all-well plot
                    text= ['']*32 + [maxRow[r+(r*11)+c]]+['']*7, # inserts max value onto plot at 33rd point; looks nice here
                    textposition='top left',
                    textfont=dict(
                        family="sans serif",
                        size=8,
                        color="black")), # want to stand out
                    row=r+1,
                    col=c+1)
        figSub.update_layout(
            height=800,
            width=1000,
            showlegend=False,
            title_text='Cycle vs Fluorescence ({})'.format(self.probe))
        return figSub.show()
                


In [4]:
# instantiate probes
# probeCy5 = Fluor('Cy5')
# probeFAM = Fluor('FAM')
# probeHEX = Fluor('HEX')
probeSYBR = Fluor('SYBR')

Fluor.plotAllWells(probeSYBR)
Fluor.plotIndWells(probeSYBR)


IndexError: single positional indexer is out-of-bounds

In [ ]:
# create df from sheet name

class PrimerMatrix:
    sheet: str
    def __init__(self, sheet):
        self.sheet = sheet
    def dfCreate(self): # make a df from the excel data
        # make df from passed sheet name
        df = dfExcel[self.sheet]
        # assign first row to be column header
        df.columns = df.iloc[0]
        # drop the first row (duplicate of header)
        df = df.drop(df.index[0])
        # convert all columns to numberic
        # df = df.apply(pd.to_numeric, errors='coerce')
        # make index the first column
        # df.set_index('Cycle', inplace=True)
        # fluor values less than 1 get 0
        # df = df.where(df>=1, 0)
        return df
    def addPrimerConcToDf(self):
        df = self.dfCreate()
        primerConcs = [50, 100, 200, 400, 600, 800] # Fwd and rev primer concentrations
        # add fwd primerConc values to two rows all the way to 72
        df['F_primer'] =np.nan
        for i in range(6): # loop through rows except last two rows containing standard curve
            for j in range(6): # loop through primer concentration list
                for k in range(2): # every concentration twice
                    pos = (12*i)+(2*j)+(k+1) # translate row and column to position in df 1..96
                    df.loc[pos, 'F_primer'] = primerConcs[j] # set value to primer concentration
        # add rev primerConc values across single row for 6 rows
        df['R_primer'] =np.nan
        for m in range(6): # loop through primer concentration list
            for n in range(12): # every concentration as a row
                pos = (12*m)+(n+1) # translate row and column to position in df 1..96
                df.loc[pos, 'R_primer'] = primerConcs[m] # set value to primer concentration
        return df 

        
    

In [ ]:
# primerMatrix = PrimerMatrix('210315_Quan. Result')
primerMatrix = PrimerMatrix('Quan. Result')
# df = PrimerMatrix.dfCreate(primerMatrix)
df = PrimerMatrix.addPrimerConcToDf(primerMatrix)

# group by F and R primer concentrations and take mean
avg_across_row_fwd = df.groupby(by=['R_primer', 'F_primer'], as_index=False)['Ct'].mean()
avg_across_col_rev = df.groupby(by=['F_primer', 'R_primer'], as_index=False)['Ct'].mean()
# stringify 'F_primer' and 'R_primer' concentrations and concatenate and add to list
barHeadingsFwd = (avg_across_row_fwd['F_primer'].astype(str) + '_' + avg_across_row_fwd['R_primer'].astype(str)).tolist()
barHeadingsRev = (avg_across_col_rev['R_primer'].astype(str) + '_' + avg_across_col_rev['F_primer'].astype(str)).tolist()
figAvgFwd = px.bar(
    avg_across_row_fwd,
    y=avg_across_row_fwd['Ct'],
    x=barHeadingsFwd, #avg_across_row['F_primer'],
    labels={'x':'F Primer Concentration', 'y':'Cq'},
    color='R_primer',
    barmode='group',
    text_auto=True,
    title='Periodicity in Increasing [Fwd] ([Rev] Held Constant)')
figAvgFwd.show()

figAvgRev = px.bar(
    avg_across_col_rev,
    y=avg_across_col_rev['Ct'],
    x=barHeadingsRev, #avg_across_row['F_primer'],
    labels={'x':'R Primer Concentration', 'y':'Cq'},
    color='F_primer',
    barmode='group',
    text_auto=True,
    title='Periodicity in Increasing [Rev] ([Fwd] Held Constant)')
figAvgRev.show()

In [ ]:
# create a new df where 'F_primer' = 'R_primer' 
df_fwd_rev = df[df['F_primer'] == df['R_primer']]
df_fwd_rev_mean = df_fwd_rev.groupby(by=['R_primer', 'F_primer'], as_index=False)['Ct'].mean()
samsieHeadings = (df_fwd_rev_mean['F_primer'].astype(str) + '_' + df_fwd_rev_mean['R_primer'].astype(str)).tolist()

figSamsies = px.bar(
    df_fwd_rev_mean,
    y=df_fwd_rev_mean['Ct'],
    x=samsieHeadings, #avg_across_row['F_primer'],
    labels={'x':'[Fwd=Rev]', 'y':'Cq'},
    # color='F_primer',
    barmode='group',
    text_auto=True,
    title='Cq when [Fwd] = [Rev]')
figSamsies.show()

# display (df_fwd_rev_mean)


In [ ]:
# one phase decay to model [F_R] vs Cq
# Y=(Y0-Plateau)*e^(-K*X)+Plateau
# solve for Y when x=100

#these values from GraphPad
gpY0=73.29
gpPlateau=19.80
gpK=0.01120
gpX=300
gpY=(gpY0-gpPlateau)*np.exp(-gpK*gpX)+gpPlateau
# 
print(gpY)

21.657989000954068


In [ ]:
# solve for one phase decay with scipy
f_r_conc = [100, 200, 400, 600, 800] # F-R primer concentrations
cq_actual = [37.32, 25.21, 21.27, 20.225, 18.81] # Cq values


# initialGuesses = [y0, Plateau, K] 
# Y0 is the Y value when X (Conc) is zero. It is expressed in the same units as Y,
# Plateau is the Y value at infinite conc, expressed in the same units as Y.
# K is the rate constant, expressed in reciprocal of the X axis conc units. If X is in nM, then K is expressed in inverse nM.
# initialGuesses = [73.29, 19.8, 0.01120]
initialGuesses = [100, 15, 0.01]


def onePhaseDecay(X, y0, Plateau, K):
    return (y0-Plateau)*np.exp(-K*X)+Plateau

from scipy.optimize import curve_fit
popt,pcov = curve_fit(onePhaseDecay, f_r_conc, cq_actual, initialGuesses)
print(popt)
# popt in the form: [y0, Plateau, K]
# print(pcov)

cq_pred = np.empty(len(cq_actual)) #empty list to receive data
for i in range(len(f_r_conc)): #loop through concentrations to make cq prediction
    # cq_pred[i]=onePhaseDecay(f_r_conc[i], initialGuesses[0], initialGuesses[1], initialGuesses[2])
    cq_pred[i]=onePhaseDecay(f_r_conc[i], popt[0], popt[1], popt[2])
    
from sklearn.metrics import r2_score
samsiesR2score = r2_score(cq_pred, cq_actual)

figSamsies = go.Figure()
figSamsies.add_trace(
    go.Scatter(
        x=f_r_conc,
        y=cq_pred,
        name='Cq Predicted (R^2 = ' + str(round(samsiesR2score, 4)) + ')', 
        mode='lines+markers',
        line=dict(color='rgb(255, 0, 0)', width=2),
        line_shape = 'spline' # make smooth line
    ))
figSamsies.add_trace(
    go.Bar(
        x=f_r_conc,
        y=cq_actual,
        name='Cq Actual',
        textposition='auto',
        marker_color='#636EFA'
    ))
figSamsies.update_layout(title='Actual vs Predicted (One Phase Decay) Cq', xaxis_title='F-R Primer Concentration', yaxis_title='Cq')
figSamsies.show()

[7.32907339e+01 1.98007113e+01 1.11979098e-02]


In [ ]:
F_cost = 12.00
conc = 300
R_cost = 12.00
# want to minimize this (cost):
F_R_cost = (F_cost+R_cost)*conc
# rewrite in terms of x, y
# F_R_cost = (F_cost+R_cost)*conc
# conc = F_R_cost/(F_cost+R_cost)
# want to minimize this (Cq):

# print(F_R_cost)

def onePhaseDecayCost(F_R_cost, F_cost, R_cost, y0, Plateau, K):
    return (y0-Plateau)*np.exp(-K*(F_R_cost/(F_cost+R_cost)))+Plateau

print (onePhaseDecayCost(F_R_cost, F_cost, R_cost, popt[0], popt[1], popt[2]))


from scipy.optimize import minimize

# x0=
# res = minimize(onePhaseDecayCost, x0, method='nelder-mead',
            #    args=(0.5, 1.), options={'xatol': 1e-8, 'disp': True})


21.659866585312102
